# O'Hara Rudy Land model

In this notebook we will experiment with a more modern model of cardiac cells. This model [[1]][1] is a modified version of the O'Hara Rudy model [[2]][2]. It also incorporated the a model for the mechanics, known as the Land model [[3]][3].


## References
[1]: <https://doi.org/10.1021/acs.jcim.0c00201> "Llopis-Lorente et. al" 
[2]: <https://doi.org/10.1371/journal.pcbi.1002061> "OHara-Rudy"
[3]: <https://doi.org/10.1016/j.yjmcc.2017.03.008> "Land"

1. Llopis-Lorente, J., Gomis-Tena, J., Cano, J., Romero, L., Saiz, J., & Trenor, B. (2020). In silico classifiers for the assessment of drug proarrhythmicity. Journal of Chemical Information and Modeling, 60(10), 5172-5187."
2. O'Hara, T., Virág, L., Varró, A., & Rudy, Y. (2011). Simulation of the undiseased human cardiac ventricular action potential: model formulation and experimental validation. PLoS computational biology, 7(5), e1002061.
3. Land, S., Park-Holohan, S. J., Smith, N. P., Dos Remedios, C. G., Kentish, J. C., & Niederer, S. A. (2017). A model of cardiac contraction based on novel measurements of tension development in human cardiomyocytes. Journal of molecular and cellular cardiology, 106, 68-83.

## Running the model

Let us start by importing the necessary modules

In [ ]:
import utils
import matplotlib.pyplot as plt

load the model

In [ ]:
model_name = "ORdmm_Land"
model = utils.load_model(model_name)

solve it for 1000 ms with a timestep of 0.1 ms

In [ ]:
t_start = 0.0
t_end = 1000.0 # ms
dt = 0.1 # 0.1 ms
solution = model.solve(t_start=t_start, t_end=t_end, dt=dt)

and plot the voltage and intracellular calcium concentration

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(8, 8))
ax[0].plot(solution.time, solution["v"])
ax[1].plot(solution.time, solution["cai"])
ax[0].set_ylabel("Volutage [mV]")
ax[1].set_ylabel("Caclium [mM]")
ax[1].set_xlabel("ms")
plt.show()

To see the available state variables to use do the following

In [ ]:
# Print the states and the default values to find exact keys
initial_states = model.default_initial_states()
print(f"{initial_states = }")

When actually solving the model we need the provide the states as a numpy array. You can get the default initial states as arrays using

In [ ]:
u0 = model.initial_state_values()
print(u0)

But you can also convert the dictionary of values to a numpy array where the values are in the correct order

In [ ]:
print(model.state_dict_to_array(initial_states))

To solve the model for multiple beats we can update the initial states in each iteration as follows

In [ ]:
num_beats = 10
dt = 0.02
t_start = 0.0
t_end = 1000.0
for i in range(num_beats):
    sol = model.solve(t_start=t_start, t_end=t_end, dt=dt, u0=u0)
    u0 = sol.u[-1, :]

and we can see that by running the model for more beats it is actually changing quite a bit

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(8, 8))
ax[0].plot(solution.time, solution["v"], label="beat 1")
ax[1].plot(solution.time, solution["cai"], label="beat 1")
ax[0].plot(sol.time, sol["v"], label="beat 10")
ax[1].plot(sol.time, sol["cai"], label="beat 10")
ax[0].set_ylabel("Volutage [mV]")
ax[1].set_ylabel("Caclium [mM]")
ax[1].set_xlabel("ms")
plt.show()

The reason for this is that we have not yet reached a steady state

## Exercise
Try to find out how many beats do you need to use before reaching steady state by plotting the solutions for different beats

### Double click for solution

<!--

fig, ax = plt.subplots(2, 1, sharex=True, figsize=(8, 8))

num_beats = 150
u0 = model.initial_state_values()
for i in range(num_beats):
    sol = model.solve(t_start=t_start, t_end=t_end, dt=dt, u0=u0)
    u0 = sol.u[-1, :]
    # Plot every 20th beat
    if i % 20 == 0:
        ax[0].plot(sol.time, sol["v"], label=f"beat {i}")
        ax[1].plot(sol.time, sol["cai"], label=f"beat {i}")

ax[0].legend()
ax[0].set_ylabel("Volutage [mV]")
ax[1].set_ylabel("Caclium [mM]")
ax[1].set_xlabel("ms")
plt.show()
# Solution seems to be fairly stable after beat 100

-->

## Extracting other values from the model
The voltage and intracellular calcium concentration are examples of state variables in the mode. However, in the model we also have other expressions that are not states, for example currents, that are interesting to plot. We refer to these expressions as monitored expressions. We can list the names of the monitored expressions in the model using the following code

In [ ]:
print(sol.montior_keys())
print([n for n in sol.montior_keys() if n.startswith("I")])

One example could be the `IKr` current. This current is very important to control the duration of the action potential and the channel where this current flow is a common channel to target when designing drugs which we will look at in another notebook.

To plot this current we can do the following

In [ ]:
IKr = sol.monitored("IKr")
fig, ax = plt.subplots()
ax.plot(sol.time, IKr)
plt.show()

As mentioned in the very beginning, the model also has a mechanical component. In short, release of calcium in the cells makes the call contract and this in turn generate an active tension. From this model, the active tension is called `Ta` which is also one of the monitored expressions.

In [ ]:
Ta = sol.monitored("Ta")
fig, ax = plt.subplots()
ax.plot(sol.time, Ta)
ax.set_ylabel("Pa")
plt.show()

The shape of `Ta` is very similar to the intracellular calcium concentration (because it is very tightly coupled to the release of calcium inside the cell). However, the unit of `Ta` is pascal, meaning that it reflect the amount of stress generated by the active contraction.  

## Exercise

In this exercise we will look at the different currents in this model. The name of the current are 

In [ ]:
current_names = [
    'INa', 'INaL', 'Ito', 'ICaL', 'ICaNa', 'ICaK', 'IKr', 'IKs', 'IK1', 'INaCa_i', 
    'INaCa_ss', 'INaK', 'IKb', 'INab', 'ICab', 'IpCa', 'Isac_P_ns', 'Isac_P_k', 'Istim'
]

Plot all the currents in on figure in order to see which currents are active in which part of the action potential.
Try to sum up all the contributions of the currents into a new array and compare this with the negative membrane potential.

### Double click for solution

<!--

import numpy as np

currents = {name: sol.monitored(name) for name in current_names}

fig, ax = plt.subplots(2, 1, figsize=(12, 8))
new_arr = np.zeros_like(sol.time)
for name, arr in currents.items():
    ax[0].plot(sol.time, arr, label=name)
    new_arr -= arr
ax[0].set_ylim(-2.0, 2.0)
ax[1].plot(sol.time, np.cumsum(new_arr) * dt + initial_states["v"], label="Sum of currents")
ax[1].plot(sol.time, sol["v"], 'k--', label="Voltage")
plt.show()

-->